In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.io import imread
from itertools import permutations
from os.path import join
from os import listdir
import pickle

from keras import Model
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, LeakyReLU
from keras.losses import binary_crossentropy
from keras.utils.generic_utils import get_custom_objects
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
from keras import backend as K

## Model

In [ ]:
def build_model(size, start_neurons=8):
    input_layer = Input(size + (1,))
    
    conv1 = Conv2D(start_neurons*1,(3,3), padding="same")(input_layer)
    conv1 = LeakyReLU(alpha=0.1)(conv1)
    conv1 = Conv2D(start_neurons*1,(3,3), padding="same")(conv1)
    conv1 = LeakyReLU(alpha=0.1)(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons*2,(3,3), padding="same")(pool1)
    conv2 = LeakyReLU(alpha=0.1)(conv2)
    conv2 = Conv2D(start_neurons*2,(3,3), padding="same")(conv2)
    conv2 = LeakyReLU(alpha=0.1)(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons*4,(3,3), padding="same")(pool2)
    conv3 = LeakyReLU(alpha=0.1)(conv3)
    conv3 = Conv2D(start_neurons*4,(3,3), padding="same")(conv3)
    conv3 = LeakyReLU(alpha=0.1)(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(start_neurons*8,(3,3), padding="same")(pool3)
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    conv4 = Conv2D(start_neurons*8,(3,3), padding="same")(conv4)
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    conv5 = Conv2D(start_neurons*16,(3,3), padding="same")(pool4)
    conv5 = LeakyReLU(alpha=0.1)(conv5)
    conv5 = Conv2D(start_neurons*16,(3,3), padding="same")(conv5)
    conv5 = LeakyReLU(alpha=0.1)(conv5)
    pool5 = MaxPooling2D((2, 2))(conv5)
    pool5 = Dropout(0.5)(pool5)

    # Middle
    convm = Conv2D(start_neurons*32,(3,3), padding="same")(pool5)
    convm = LeakyReLU(alpha=0.1)(convm)
    convm = Conv2D(start_neurons*32,(3,3), padding="same")(convm)
    convm = LeakyReLU(alpha=0.1)(convm)

    deconv5 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv5 = concatenate([deconv5, conv5])
    uconv5 = Dropout(0.5)(uconv5)
    uconv5 = Conv2D(start_neurons*16,(3,3), padding="same")(uconv5)
    uconv5 = LeakyReLU(alpha=0.1)(uconv5)
    uconv5 = Conv2D(start_neurons*16,(3,3), padding="same")(uconv5)
    uconv5 = LeakyReLU(alpha=0.1)(uconv5)

    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv5)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons*8,(3,3), padding="same")(uconv4)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)
    uconv4 = Conv2D(start_neurons*8,(3,3), padding="same")(uconv4)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)

    deconv3 = Conv2DTranspose(start_neurons*4,(3,3),strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons*4,(3,3), padding="same")(uconv3)
    uconv3 = LeakyReLU(alpha=0.1)(uconv3)
    uconv3 = Conv2D(start_neurons*4,(3,3), padding="same")(uconv3)
    uconv3 = LeakyReLU(alpha=0.1)(uconv3)

    deconv2 = Conv2DTranspose(start_neurons*2,(3,3),strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons*2,(3,3), padding="same")(uconv2)
    uconv2 = LeakyReLU(alpha=0.1)(uconv2)
    uconv2 = Conv2D(start_neurons*2,(3,3), padding="same")(uconv2)
    uconv2 = LeakyReLU(alpha=0.1)(uconv2)

    deconv1 = Conv2DTranspose(start_neurons*1,(3,3),strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons*1,(3,3), padding="same")(uconv1)
    uconv1 = LeakyReLU(alpha=0.1)(uconv1)
    uconv1 = Conv2D(start_neurons*1,(3,3), padding="same")(uconv1)
    uconv1 = LeakyReLU(alpha=0.1)(uconv1)

    uncov1 = Dropout(0.5)(uconv1)
    output_layer = Conv2D(1,(1,1), padding="same", activation="sigmoid")(uconv1)
    
    model = Model(input_layer, output_layer)
    return model

In [ ]:
def PSNR(y_true, y_pred):
    max_pixel = 255
    return 10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true))))/K.log(10.0)

get_custom_objects().update({'PSNR': PSNR})

In [ ]:
def SSIM(y_true, y_pred):
    return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

get_custom_objects().update({'SSIM': SSIM})

In [ ]:
def MSE(img1, img2):
    shape = np.array(img1.shape).prod()
    res = ((img1-img2)**2).sum()/shape
    if (res == 0):
        raise ValueError
    else:
        return res

def compute_psnr(img_pred, img_gt):
    img_pred = img_pred.astype('float64')
    img_gt = img_gt.astype('float64')
    max_pixel = 255
    return 10*np.log10(max_pixel**2/MSE(img_pred, img_gt))

## Data

In [ ]:
data_dir = 'drive/MyDrive/CV/Coursework/images'
dirs = listdir(data_dir)

combinations = []

for dir in dirs:
  cur_dir = join(data_dir, dir)
  files = listdir(cur_dir)

  for i,file in enumerate(files):
    files[i] = join(cur_dir, file)

  combinations.extend(list(permutations(files, 2)))

train, valid = train_test_split(combinations, test_size=0.1, shuffle=True)

In [ ]:
def datagen(pairs, size, batch_size):
    x = np.empty((batch_size,)+size)
    y = np.empty((batch_size,)+size)

    batch_ind = 0
    epoch_ind = 0
    n_samples = len(pairs)

    while True:
        train_img = imread(pairs[epoch_ind][0], as_gray=True)
        target_img = imread(pairs[epoch_ind][1], as_gray=True)

        x[batch_ind] = resize(train_img, size)
        y[batch_ind] = resize(target_img, size)

        batch_ind += 1
        epoch_ind += 1

        if batch_ind == batch_size:
            batch_ind = 0
            yield x.reshape(x.shape+(1,)), y.reshape(y.shape+(1,))

        if epoch_ind == n_samples:
            epoch_ind = 0

## Train

In [ ]:
size = (256,1024)
batch_size = 16
epochs = 30

train_datagen = datagen(train, size, batch_size)
valid_datagen = datagen(valid, size, batch_size)

steps_per_train_epoch = len(train)//batch_size
steps_per_valid_epoch = len(valid)//batch_size

model = build_model(size)

model.compile(loss='mse', optimizer=Adam(), metrics=['PSNR', 'SSIM'])

history = model.fit(train_datagen, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    verbose=1,
                    validation_data=valid_datagen,
                    steps_per_epoch=steps_per_train_epoch,
                    validation_steps=steps_per_valid_epoch
                    )

model_name = 'cr_removing'
model_dir = 'drive/MyDrive/CV/Coursework/models'
stats_dir = 'drive/MyDrive/CV/Coursework/models'

model.save(join(model_dir, 'model_{}.hdf5'.format(model_name)))
with open(join(stats_dir, 'stats_{}.pickle'.format(model_name)), 'wb') as f:
    pickle.dump(history.history, f)

Epoch 1/30
164/164 [==============================] - 1894s 12s/step - loss: 0.0381 - PSNR: 62.9484 - SSIM: 0.3252 - val_loss: 0.0188 - val_PSNR: 65.5916 - val_SSIM: 0.5204
Epoch 2/30
164/164 [==============================] - 960s 6s/step - loss: 0.0191 - PSNR: 65.6294 - SSIM: 0.4832 - val_loss: 0.0162 - val_PSNR: 66.3843 - val_SSIM: 0.5609
Epoch 3/30
164/164 [==============================] - 939s 6s/step - loss: 0.0182 - PSNR: 65.8910 - SSIM: 0.5007 - val_loss: 0.0159 - val_PSNR: 66.3565 - val_SSIM: 0.5659
Epoch 4/30
164/164 [==============================] - 929s 6s/step - loss: 0.0178 - PSNR: 65.9584 - SSIM: 0.5090 - val_loss: 0.0157 - val_PSNR: 66.5073 - val_SSIM: 0.5698
Epoch 5/30
164/164 [==============================] - 933s 6s/step - loss: 0.0177 - PSNR: 66.0196 - SSIM: 0.5140 - val_loss: 0.0152 - val_PSNR: 66.6971 - val_SSIM: 0.5730
Epoch 6/30
164/164 [==============================] - 941s 6s/step - loss: 0.0175 - PSNR: 66.0709 - SSIM: 0.5185 - val_loss: 0.0153 - val_PSNR: